# Image processing

## Load an image to numpy

In [270]:
from typing import List
import PIL
from PIL import Image, ImagePalette
import numpy as np
print('Pillow Version:', PIL.__version__)
import pandas as pd

Pillow Version: 8.1.0


Let's create a helper to manipulate the image with a palette

In [271]:
class ImageReaderHelper:
    def __init__(self, name: str):
        self.name = name
    
    def load(self):
        self.img = Image.open(f'{self.name}.png').convert('P')
        self.palette = self.img.getpalette()
        self.img_arr = np.asarray(self.img)
        self.palette_arr = np.asarray(self.palette).reshape(256, 3)
        print(self.palette_arr)
        print(f"{self.name} shape: {self.img_arr.shape} dtype: {self.img_arr.dtype}")
        print(f"{self.name} palette: {self.palette_arr.shape} dtype: {self.palette_arr.dtype}")
        print('Unique colors used from palette in the image')
        self.colors_used = np.unique(self.img_arr)
        print(self.colors_used)
        print('Unique RGB colors used')
        self.min_color_idx = np.amin(self.colors_used)
        self.max_color_idx = np.amax(self.colors_used)
        print(f'Color index range: {self.min_color_idx} to {self.max_color_idx}')
        
        
    def print_palette_info(self):
        rgb_colors_used = self.palette_arr[self.min_color_idx:self.max_color_idx+1]
        print(rgb_colors_used)
        color_id, color_frequency = np.unique(self.img_arr, return_counts = True)
        print('Color frequency')
        print(color_frequency)
        
    def get_pixels(self):
        return self.img_arr
    
    def get_colors(self):
        return self.palette_arr
    
    def get_shuffle_colors(self):
        new_palette_arr = self.palette_arr[self.min_color_idx:self.max_color_idx+1].copy()
        np.random.shuffle(new_palette_arr)
        return new_palette_arr


In [272]:
veg2020 = ImageReaderHelper('vegetation2020')
veg2020.load()

[[254 254 254]
 [  2   2   2]
 [232 232 232]
 [215 215 215]
 [ 23  23  23]
 [ 55  71  97]
 [199 199 200]
 [183 183 183]
 [ 39  39  39]
 [167 167 168]
 [ 55  55  55]
 [135 135 136]
 [151 151 151]
 [ 71  71  71]
 [119 119 119]
 [ 87  87  87]
 [103 103 103]
 [ 72  87 110]
 [142 151 165]
 [107 119 138]
 [ 87 101 122]
 [180 186 195]
 [122 132 150]
 [188 194 202]
 [155 163 176]
 [217 220 225]
 [ 97 109 130]
 [222 225 229]
 [ 95 107 128]
 [ 64  79 104]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0 

## Understand the palette

We can check the how many colors from the palette are been used in the entire image. For vegetation2020, it is a palette of 30 colors.

In [273]:
veg2020.print_palette_info()

[[254 254 254]
 [  2   2   2]
 [232 232 232]
 [215 215 215]
 [ 23  23  23]
 [ 55  71  97]
 [199 199 200]
 [183 183 183]
 [ 39  39  39]
 [167 167 168]
 [ 55  55  55]
 [135 135 136]
 [151 151 151]
 [ 71  71  71]
 [119 119 119]
 [ 87  87  87]
 [103 103 103]
 [ 72  87 110]
 [142 151 165]
 [107 119 138]
 [ 87 101 122]
 [180 186 195]
 [122 132 150]
 [188 194 202]
 [155 163 176]
 [217 220 225]
 [ 97 109 130]
 [222 225 229]
 [ 95 107 128]
 [ 64  79 104]]
Color frequency
[1882788   52211   17195   12337   10959   10353   10175    8440    8384
    7944    7288    7085    6974    6744    6633    6514    6427     846
     789     674     526     507     474     363     352     305     118
      94      62      39]


## Change the palette

In [274]:
class ImageWriteHelper:
    def __init__(self, name: str):
        self.name = name
        
    def set_colors(self,colors: List):
        # Palette would be provided as [(R, G, B), ..]
        self.colors = colors
        
    def set_pixels(self,pixels: List):
        self.pixels = np.asarray(pixels).astype(np.uint8)
    
    def _create_image_palette(self):
        # The list must be aligned by channel (All R values must be contiguous in the list before G and B values.)
        colors = np.asarray(self.colors).astype(np.uint8).flatten()
        r_colors = colors[0::3]
        g_colors = colors[1::3]
        b_colors = colors[2::3]
        cols_palette = np.concatenate((r_colors, g_colors, b_colors), axis=None).tolist()
        img_palette = ImagePalette.ImagePalette(mode='RGB', palette=cols_palette, size=len(cols_palette))
        return img_palette
    
    def _create_image(self):
        new_img = Image.fromarray(self.pixels, 'P')
        new_img.putpalette(self._create_image_palette())
        return new_img
    
    def save(self):
        self._create_image().save(f"{self.name}.png")
    

In [275]:
img_rand_palette = ImageWriteHelper('img-exp-random-palette')
img_rand_palette.set_pixels(veg2020.get_pixels())
img_rand_palette.set_colors(veg2020.get_shuffle_colors())
img_rand_palette.save()

## Create a tri-colors grid image

In [276]:
col_white = (255, 255, 255)
col_black = (0, 0, 0)
col_grey = (85, 86, 87)

In [277]:
img_grid = ImageWriteHelper('img-exp-grid')
img_grid.set_colors([col_white, col_black, col_grey])
pix_grid = np.ones((1080, 1920))
for i in range(1080):
    for j in range(1920):
       pix_grid[i, j]= 1 if i % 10 > 5 and j % 10 > 5 else 0
print(pix_grid)
img_grid.set_pixels(pix_grid)
img_grid.save()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]
